In [2]:
import numpy as np
import cvxpy as cp
import pandas as pd

df_X = pd.read_csv('Xsvm.csv',header=None)
df_Y = pd.read_csv('ysvm.csv',header=None)
X = np.array(df_X,dtype=np.float64)
Y = np.array(df_Y,dtype=np.float64)
print(X.shape,Y.shape)

(500, 2) (500, 1)


In [52]:
# Convex Optimization
a = cp.Variable(len(Y))
R1 = cp.matmul(cp.diag(a),Y)
R2 = cp.matmul(X.T,R1)
R4 = cp.norm(R2)**2
R4.shape

()

In [53]:
P1 = cp.sum(a)
Const1 = P1 - 0.5*R4
# Const1 = np.reshape(Const1,(1,))

In [54]:
Const2 = cp.matmul(a.T,Y)
Const3 = [0<=a,Const2 == 0]
obj = cp.Maximize(Const1)
prob = cp.Problem(obj, Const3)
prob.solve(verbose=True)

----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 2019, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 508, constraints m = 517
Cones:	primal zero / dual free vars: 8
	linear vars: 500
	soc vars: 3, soc blks: 1
	sd vars: 6, sd blks: 1
Setup time: 2.70e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 9.05e+20  1.11e+20  1.00e+00 -2.50e+21 -0.00e+00  1.88e+21  3.16e-02 
   100| 1.61e-01  9.62e-02  1.18e-02 -5.53e+01 -5.66e+01  5.56e-15  6.67e+00 
   200| 7.2

69.43714789794153

In [55]:
print(a.value)

[ 4.83467762e-09 -5.34278007e-08 -5.33516779e-08 -2.96424101e-08
 -2.51841332e-08 -9.17316098e-09 -4.62985482e-08 -5.02753807e-08
  5.48279185e-09  8.48038289e-09 -5.48366796e-08 -4.81224878e-08
 -5.01059763e-08  4.48947008e-09 -1.77338733e-08  1.17043846e-08
 -2.62100043e-08 -6.58450040e-08 -5.48600575e-08 -3.88494931e-08
 -2.44369835e-08 -3.36227825e-08 -3.49887787e-08  1.78975847e-08
 -2.17271196e-08 -2.40824017e-08 -4.47661665e-08 -2.09578283e-08
 -5.16007357e-08 -6.79296622e-08  7.31613688e-09 -4.36803757e-08
 -2.23333941e-08  3.53501499e+00 -5.87796878e-09 -1.91495304e-08
 -6.66801882e-08 -4.41998141e-08 -6.21596908e-08 -3.53138700e-09
  1.59385980e-08  1.01913439e-08 -8.64232779e-09 -2.46414285e-08
 -6.50243254e-08 -8.36884672e-09 -2.06723145e-08 -2.42944619e-08
 -1.91047559e-08 -2.96337163e-08 -1.69921486e-08  9.23254453e-09
  2.31990071e-09 -5.67251652e-08  1.21909811e-08 -7.71440365e-09
 -3.25124214e-09 -3.57043471e-08 -3.62105913e-08 -2.39880193e-08
 -1.84396568e-09 -1.46824

In [70]:
A = (np.array(a.value)).reshape(500,1)

In [73]:
W = np.zeros((2,))
for i in range(len(Y)):
    W += A[i]*Y[i]*(X[i].T)
    if(A[i]>1e-4):
        print(i)

33
281
469


In [72]:
print(W)

[-0.11843106 11.78390426]


In [77]:
W0 = (1/Y[281]) - np.dot(W,X[281])
print(W0)

[0.10229339]


In [84]:
Test = np.array([[2,0.5],[0.8,0.7],[1.58,1.33],[0.008, 0.001]])

for i in range(len(Test)):
    est = np.sign(np.dot(W,Test[i])+W0)
    print(Test[i],est)

[2.  0.5] [1.]
[0.8 0.7] [1.]
[1.58 1.33] [1.]
[0.008 0.001] [1.]


In [85]:
# Verification
from sklearn import svm

clf = svm.SVC()
clf.fit(X,Y)

/home/legion/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [86]:
clf.predict(Test)

array([1., 1., 1., 1.])